# Harvest items from a search in RecordSearch

RecordSearch, the National Archives of Australia's online database doesn't currently have an option for downloading machine-readable data. So to get collection metadata in a structured form, we have to resort of screen-scraping. All the screen-scraping code is contained in the [recordsearch_tools](https://github.com/wragge/recordsearch_tools) package.

This notebook saves data from a search for items in RecordSearch. An item in RecordSearch is usually a file, but might also be a photo, a volume, or some other sort of container. 

Using this notebook you can save the results of a search for items as a CSV-formatted file for easy download and analysis.

<div class="alert alert-block alert-warning">
<p>If you haven't used one of these notebooks before, they're basically web pages in which you can write, edit, and run live code. They're meant to encourage experimentation, so don't feel nervous. Just try running a few cells and see what happens!</p>

<p>
    Some tips:
    <ul>
        <li>Code cells have boxes around them.</li>
        <li>To run a code cell click on the cell and then hit <b>Shift+Enter</b>. The <b>Shift+Enter</b> combo will also move you to the next cell, so it's a quick way to work through the notebook.</li>
        <li>While a cell is running a <b>*</b> appears in the square brackets next to the cell. Once the cell has finished running the asterix will be replaced with a number.</li>
        <li>In most cases you'll want to start from the top of notebook and work your way down running each cell in turn. Later cells might depend on the results of earlier ones.</li>
        <li>To edit a code cell, just click on it and type stuff. Remember to run the cell once you've finished editing.</li>
    </ul>
</p>

<p><b>Is this thing on?</b> If you can't edit or run any of the code cells, you might be viewing a static (read only) version of this notebook. Click here to <a href="">load a <b>live</b> version</a> running on Binder.</p>

</div>

In [176]:
from tqdm.auto import tqdm
from tinydb import TinyDB
from pathlib import Path
import math
import time
from datetime import datetime
from recordsearch_tools.client import RSSearchClient, TooManyError
from slugify import slugify
import pandas as pd
from pandas.io.json import json_normalize
import json
from IPython.display import display, FileLink

# Make sure the 'data' directory exists
Path('data').mkdir(exist_ok=True)

## Available search parameters

The available search parameters are the same as those in RecordSearch's Advanced Search form. There's lots of them, but you'll probably only end up using a few like `kw` and `series`. Note that you can use \* for wildcard searches as you can in the web interface. So setting `kw` to 'wragge\*' will find both 'wragge' and 'wragges'.

* `kw` – string containing keywords to search for
* `kw_options` – how to interpret `kw`, possible values are:
    * 'ALL' – return results containing all of the keywords (default)
    * 'ANY' – return results containg any of the keywords
    * 'EXACT' – treat `kw` as a phrase rather than a list of words
* `kw_exclude` – string containing keywords to exclude from search
* `kw_exclude_options` – how to interpret `kw_exclude`, possible values are:
    * 'ALL' – exclude results containing all of the keywords (default)
    * 'ANY' – exclude results containg any of the keywords
    * 'EXACT' – treat `kw_exact` as a phrase rather than a list of words
* `search_notes` – set to 'on' to search item notes as well as metadata
* `series` – search for items in this series
* `series_exclude` – exclude items from this series
* `control` – search for items matching this control symbol
* `control_exclude` – exclude items matching this control symbol
* `barcode` – search for items with this barcode number
* `date_from` – search for items with a date (year) greater than or equal to this, eg. '1935'
* `date_to` – search for items with a date (year) less than or equal to this
* `formats` – limit search to items in a particular format, see possible values below
* `formats_exclude` – exclude items in a particular format, see possible values below
* `locations` – limit search to items held in a particular location, see possible values below
* `locations_exclude` – exclude items held in a particular location, see possible values below
* `access` – limit to items with a particular access status, see possible values below
* `access_exclude` – exclude items with a particular access status, see possible values below
* `digital` – set to 'on' to limit to items that are digitised


Possible values for `formats` and `formats_exclude`: 

* 'Paper files and documents'
* 'Index cards'
* 'Bound volumes'
* 'Cartographic records'
* 'Photographs'
* 'Microforms'
* 'Audio-visual records'
* 'Audio records'
* 'Electronic records'
* '3-dimensional records'
* 'Scientific specimens'
* 'Textiles'

Possible values for `locations` and `locations_exclude`:

* 'NAT, ACT'
* 'Adelaide'
* 'Australian War Memorial'
* 'Brisbane'
* 'Darwin'
* 'Hobart'
* 'Melbourne'
* 'Perth'
* 'Sydney'

Possible values for `access` and `access_exclude`:

* 'OPEN'
* 'OWE'
* 'CLOSED'
* 'NYE'


## Create a search

Once you've decided on your parameters you can use them to create a search. For example, if we wanted to find all items that included the word 'wragge' and were digitised, our parameters would be:

* `kw='wragge'`
* `digital='on'`

In [194]:
# Create a search client
c = RSSearchClient()

In [195]:
# Feed the search parameters to the client and save the results
results =  c.search(kw='wragge', digital='on')

Now we can have a look to see how many results there are in the complete results set.

In [196]:
# Display total results
results['total_results']

'33'

Note that the search client only gets one page of results (containing 20 items) at a time. You can check this.

In [200]:
# How many results do we actually have
len(results['results'])

20

Let's have a look at the first item.

In [199]:
results['results'][0]

{'series': 'A2479',
 'control_symbol': '17/1306',
 'title': 'The Wragge Estate. Property for sale.',
 'access_status': 'Open',
 'location': 'Canberra',
 'contents_dates': {'date_str': '1917 - 1917',
  'start_date': '1917',
  'end_date': '1917'},
 'digitised_status': True,
 'digitised_pages': 4,
 'identifier': '149309'}

## Changing how your search results are delivered

There are some additional parameters that affect the way the search results are delivered. We'll use some of these to harvest the complete results set.

* `page` – return a specific page of research results
* `sort` – return results in a specified order, possible values:
    * 1 – series and control symbol
    * 3 – title
    * 5 – start date
    * 7 – digitised items first
    * 12 – items with pdfs first
    * 9 – barcode
    * 11 – audio visual items first
* `digitised` – set to `True` (default) or `False` to control whether to include the number of pages in each digitised file (if `True`, extra requests are made to get the info which slows things down a bit)

So to get the second page of results from the search above.

In [29]:
results =  c.search(kw='wragge', digital='on', page=2)

The first item in our result set should be different, because it's coming from the second page of results.

In [30]:
results['results'][0]

{'series': 'A6770',
 'control_symbol': 'WRAGGE K C',
 'title': 'WRAGGE KEITH CLEMENT : Service Number - B/2680 : Date of birth - 16 Jan 1922 : Place of birth - BRISBANE QLD : Place of enlistment - BRISBANE : Next of Kin - RUPERT',
 'access_status': 'Open',
 'location': 'Canberra',
 'contents_dates': {'date_str': '1939 - 1948',
  'start_date': '1939',
  'end_date': '1948'},
 'digitised_status': True,
 'digitised_pages': 2,
 'identifier': '4523493'}

## Harvesting a complete set of (less than 20,000) results

Ok, we've learnt how to create a search and get back some data, but only getting the first 20 results is not so useful. What if our search contains hundreds or thousands of items? How do we get them all?

To save everything, we have to loop through each page in the result set, saving the results as we go. The functions below do just that.

But wait! You might have noticed that RecordSearch only displays results for searches that return fewer than 20,000 items. Because the screen scraper is just extracting details from the RecordSearch web pages, the 20,000 limit applies here as well. If your search has more than 20,000 results, you'll need to narrow it down using additional parameters.

The main function below is `harvest_items()`. You just give it any of the search parameters listed above. It will loop through all the pages in the result set, saving the items to a simple JSON database using TinyDB.

The database will be created in the `data` directory. It's name will include a timestamp, identifying the time at which the harvest was started. For example `db-items-1567492794.json`. There are more functions for using and managing the db files below.

In [183]:
def get_total_results(client, **kwargs):
    '''
    Get the total number of results returned by a search.
    '''
    try:
        # Get the first page of results, passing digitised=Flase to speed things up
        results = client.search(digitised=False, **kwargs)
        
        # Get the total number of results
        total = results['total_results']
        
    # Uh oh there are more than 20,000 results
    except TooManyError:
        print('There are more than 20,000 results.')
        total = None
    return total

def harvest_items(start=1, db_path=None, **kwargs):
    '''
    Harvest items from a search and save them to a database.
    Supply any of the search parameters listed above.
    '''
    # Initiate the client
    client = RSSearchClient()
    
    # Get the total number of results returned by this search
    total_results = get_total_results(client, **kwargs)
    
    # If the number of results is between 1 and 20,000 we can harvest!
    if total_results:
        
        # Calculate the number of results pages
        total_pages = math.ceil(int(total_results) / client.results_per_page)
        
        # We're creating a new db
        if not db_path:
            
            # Get the current timestamp
            timestamp = int(time.time())
            
            # Create a new db using the timestamp as a name
            db = TinyDB(Path(f'data/db-items-{timestamp}.json'))
            
            # Save the details of this harvest to the 'meta' table of the database
            # This keeps the query metadata with the results, and helps us to restart the harvest if necessary.
            db.table('meta').insert({
                'timestamp': timestamp, 
                'total_results': int(total_results), 
                'total_pages': total_pages, 
                'results_per_page': client.results_per_page,
                'params': kwargs
            })
        else:
            
            # If we have an existing db, open it
            db = TinyDB(db_path)
            
        # Loop through the range of pages
        for page in tqdm(range(start, total_pages + 1), unit='page', desc='Pages:'):
            
            # Get results from each page
            # Note that sort is set to 9 (barcode) to make sure the pages stay in the same order
            # If we don't set the sort param we can end up getting duplicates and missing records
            items = client.search(sort=9, page=page, **kwargs)
            
            # Save the results from this page to the db
            db.table('items').insert_multiple(items['results'])       

In [ ]:
# Create a harvest!
harvest_items(kw='wragge')

## Managing harvests

If you're doing a large harvest, you might find that it fails part way through. You might also want to check on the details of a past harvest, or even reharvest a query to see if anything new has been added. Because we've saved the harvest metadata and results into a TinyDB database, it's easy to perform some basic checks and management tasks.

There are three main functions defined below:

* `harvest_report()` – prints basic details of a harvest
* `harvest_restart()` – restarts a failed harvest
* `reharvest_items()` – creates a new harvest using the query settings of an existing harvest

In each case you can specify the path to an existing harvest database, something like `data/db-items-1567480717.json`. If you don't specify a database, the function will assume you want the most recent.

Here's an example of the output from `harvest_report()`.

```
Harvest started: 2019-09-03 15:21:11
Items harvested: 200 of 200

{'timestamp': 1567488071,
 'total_results': 200,
 'total_pages': 10,
 'results_per_page': 20,
 'params': {'kw': 'wragge'}}
```

In [185]:
def get_latest_db():
    '''
    Get the database created by the most recent harvest.
    '''
    p = Path('data')
    dbs = sorted(list(p.glob('db-items-[0-9]*.json')))
    try:
        latest = dbs[-1]
    except IndexError:
        print('No databases')
        latest = None
    return latest

def get_db(db_path):
    '''
    Get a harvest database.
    If db_path is supplied then return that db.
    If not, then return the most recently created db.
    '''
    db = None
    if not db_path:
        db_path = get_latest_db()
    if db_path:  
        db = TinyDB(db_path)
    return db

def harvest_report(db_path=None):
    '''
    Print a report of the specified harvest.
    If db_path is not supplied, display details from the most recently created harvest.
    '''
    db = get_db(db_path)
    if db is not None:
        meta = db.table('meta').all()[0]
        date = datetime.fromtimestamp(meta['timestamp']).strftime('%Y-%m-%d %H:%M:%S')
        items_harvested = len(db.table('items').all())
        print(f'Harvest started: {date}')
        print(f'Items harvested: {items_harvested} of {meta["total_results"]}\n')
        display(meta)
    
def harvest_restart(db_path=None):
    '''
    Attempt to restart the specified harvest.
    If db_path is not supplied, restart the most recently created harvest.
    '''
    db = get_db(db_path)
    if db is not None:
        meta = db.table('meta').all()[0]
        items_harvested = len(db.table('items').all())
        pages_harvested = math.ceil(items_harvested / meta['results_per_page'])
        if pages_harvested < meta['total_pages']:
            start = pages_harvested + 1
            harvest_items(db_path=db_path, start=start, **meta['params'])
        else:
            print('Harvest complete')
            
def reharvest_items(db_path=None):
    '''
    Harvest items using the parameters of the specified db.
    If db_path is not supplied, use the most recently created harvest.
    '''
    db = get_db(db_path)
    if db is not None:
        meta = db.table('meta').all()[0]
        harvest_items(**meta['params'])

In [186]:
# Display details of the most recent harvest
# Optionally, supply the path to an existing db, eg: harvest_report('data/db-items-1567480968.json')
harvest_report()

Harvest started: 2019-09-03 16:40:15
Items harvested: 200 of 200



{'timestamp': 1567492815,
 'total_results': 200,
 'total_pages': 10,
 'results_per_page': 20,
 'params': {'kw': 'wragge'}}

In [ ]:
# Restart the most recent harvest
# Optionally, supply the path to an existing db, eg: harvest_restart('data/db-items-1567480968.json')
harvest_restart()

In [ ]:
# Create a new harvest using the parameters of the most recent harvest
# Optionally, supply the path to an existing db, eg: reharvest_items('data/db-items-1567480968.json')
reharvest_items()

## Saving a harvest

Although your harvest is already saved in a TinyDB database, you might want to convert it to a simpler format for download and analysis. The functions below provide two options:

* `save_harvest_as_json()` – save the harvested items as a JSON file
* `save_harvest_as_csv()` – save the harvested items as a CSV file

The columns in the CSV-formatted file are:

* `identifier` – the barcode number
* `series` – identifier of the series which contains the item
* `control_symbol` – individual control symbol
* `title` – title of the item
* `contents_date_str` – the contents date string as in RecordSearch
* `contents_start_date` – the first date in the contents date string converted to ISO format
* `contents_end_date` – the second date in the contents date string converted to ISO format
* `location` – where the file is held
* `access_status` – 'Closed', 'Open', 'OWE, or 'NYE
* `digitised_status` – True/False, has the file been digitised
* `digitised_pages` – number of pages in the digitised file

In [187]:
def save_harvest_as_json(db_path=None):
    '''
    Save harvested items as a json file.
    If db_path is not supplied, use the most recently created harvest.
    '''
    db = get_db(db_path)
    if db is not None:
        
        # Get harvest metadata
        meta = db.table('meta').all()[0]
        
        # Get harvested items
        items = db.table('items').all()
        
        # Set file name and path
        filename = Path(f'data/items-{meta["timestamp"]}.json')
        
        # Dump items to a JSON file
        with open(filename, 'w') as json_file:
            json.dump(items, json_file)
        
        # Display link to download
        display(FileLink(filename))
            
def save_harvest_as_csv(db_path=None):
    '''
    Save harvested items as a CSV file.
    If db_path is not supplied, use the most recently created harvest.
    '''
    db = get_db(db_path)
    if db is not None:
        
        # Get harvest metadata
        meta = db.table('meta').all()[0]
        
        # Get harvested items
        items = db.table('items').all()
        
        # Flatten the date field using json_normalise and convert the items to a dataframe
        df = pd.DataFrame(json_normalize(items))
        
        # Rename the date columns
        df.rename(columns={'contents_dates.date_str': 'contents_date_str', 'contents_dates.start_date': 'contents_start_date', 'contents_dates.end_date': 'contents_end_date'}, inplace=True)
        
        # Put the columns in a nice order
        df = df[['identifier', 'series', 'control_symbol', 'title', 'contents_date_str', 'contents_start_date', 'contents_end_date', 'location', 'access_status', 'digitised_status', 'digitised_pages']]
        
        # Set file name and path
        filename = Path(f'data/items-{meta["timestamp"]}.csv')
        
        # Save as CSV
        df.to_csv(filename, index=False)
        
        # Display link to download
        display(FileLink(filename))


In [190]:
# Save the most recent harvest as a json file
# Optionally, supply the path to an existing db, eg: save_harvest_as_json('data/db-1567480968.json')
save_harvest_as_json()

/Volumes/Workspace/mycode/glam-workbench/recordsearch/notebooks/data/items-1567492815.json

In [191]:
# Save the most recent harvest as a CSV file
# Optionally, supply the path to an existing db, eg: save_harvest_as_json('data/db-1567480968.json')
save_harvest_as_csv()

/Volumes/Workspace/mycode/glam-workbench/recordsearch/notebooks/data/items-1567492815.csv

In [ ]:
----

Created by [Tim Sherratt](https://timsherratt.org/) as part of the GLAM Workbench